In [42]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [43]:
TRAIN_TEXT_FILE_PATH = 'train_text.txt'

#!/usr/bin/env python
# -*- coding: utf-8 -*-
# vim:fileencoding=utf-8
# 

with open(TRAIN_TEXT_FILE_PATH, encoding="utf8") as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'е', 'а', 'и', 'н', 'т', 'с', 'р', 'л', 'к', 'м', 'в', 'д', 'ь', 'у', 'п', 'я', 'ы', 'з', 'б', 'ч', ',', 'г', 'й', '.', 'ж', 'ш', 'х', 'ц', 'ю', 'щ', '-', 'э', 'В', 'Э', '«', '»', 'К', 'П', 'О', 'С', '!', 'ф', 'Т', 'Д', 'Н', 'И', 'А', 'Е', 'М', ':', '–', 'ё', 'У', '5', '7', 'Г', '?', '4', '6', 'З', 'Ф', 'Л', 'Р', 'Б', 'Х', '\n', 'Ж', 'ъ', 'Ч', 'Я', 'X', 'I', 'V', '(', ')', '/']


In [44]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [45]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    while predicted_char!="." and predicted_char!="!":
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
        
    return predicted_text


In [46]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [47]:
#вот тут нейронка обучается. Если чё, можно не дожидаться до конца и остановть
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 50000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 2.8114554023742677
 дотольно тосте на не постатом стоми породно соторольно встами тольно кольно ванто с додно сашения поворосте стольцо достовостет стольно стостими на станнов водости стостольно стольно стольно с и кольно вистольно стольно и стостольно стольно вот в это потостовосто на стостомать постому вод тольно потольно стотольно пододно и датьно вантом серетость стольно кольно ни кодочения стольно потом ванно стодольно в в кашения но додит комить соводить подно и ванно соторость ванто старать столяните на вередно на середнольной подит вантом пододно стольно сересте стодод кольно подобление танно в дально состоторанить в предно стостольно стольно стольно стольно подоритольно подно с стостим в вашение стами в тольно стольно с подоние с с на вольно стово потостод сератим в валения с стоми кольцо с востольно стольно в додно стольно стодосто сотольно стовосто кольно кольцо подотрения стодорость стольно стородно пододно комак стодостить ограните стольно стольной в потольно стороль

In [ ]:
import pickle
#сохранение обученной модели

torch.save(model.state_dict(), 'model.pt')

with open('char_to_idx.pickle', 'wb') as f:
    pickle.dump(char_to_idx, f)

with open('idx_to_char.pickle', 'wb') as f:
    pickle.dump(idx_to_char, f)

In [ ]:
#сам генератор
model.load_state_dict(torch.load("model.pt"))
model.eval()


print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=200, 
    start_text='Это золотое кольцо 777 карат с голубым бриллиантом'
    )
)

Это золотое кольцо 777 карат с голубым бриллиантомя самую прочную «связь» - в это верили древние греки, соотнося необычный благородный металл со столь же таинственным ночным светилом, символом богини Артемиды. Важно: Луна должна быть непременно растущей.
